In [2]:
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import pickle

In [3]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import GradientLLM

In [4]:
import os
os.environ["GRADIENT_ACCESS_TOKEN"] = "qmo0P8rEERRVzm0XoEIvm9j41aQEu5Z2"
os.environ["GRADIENT_WORKSPACE_ID"] = "fde7dc26-fb49-4ba9-9fc3-818e15c189ae_workspace"

In [5]:
# gradient accesst token qmo0P8rEERRVzm0XoEIvm9j41aQEu5Z2

In [6]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [7]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [8]:
llm = GradientLLM(
    # `ID` listed in `$ gradient model list`
    model="6fd9d674-c159-418f-98d7-689af2b11715_model_adapter",
    # # optional: set new credentials, they default to environment variables
    # gradient_workspace_id=os.environ["GRADIENT_WORKSPACE_ID"],
    # gradient_access_token=os.environ["GRADIENT_ACCESS_TOKEN"],
    model_kwargs=dict(max_generated_token_count=400,temperature= 0.75, max_length= 3000, Stream=True),
    # callback_manager= callback_manager,
    callbacks=[StreamingStdOutCallbackHandler()]
)


In [22]:
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

In [9]:
from langchain_core.prompts import ChatPromptTemplate
template = """You are a financial expert with access to the annual report of the company.
When answering questions about the company's financial performance, prioritize information from the Financial Statements section.Considering the user's question, provide clear and concise answers from given context.
{context}

Question: {question}
Answer:
"""
# template = """You are a financial expert with access to the annual report of the company.
# When answering questions about the company's financial performance, prioritize information from the Financial Statements section.

# {context}

# **Question:** {question}

# **Answer:**

# If you can find relevant information in the annual report, please provide a clear and concise answer based on the facts.
# If you're unable to find an answer in the report, you can respond by saying:

# * "I couldn't find information related to '{question}' in the annual report."
# * "The annual report doesn't provide sufficient details to answer this question definitively."

# """


prompt = ChatPromptTemplate.from_template(template)

In [10]:
with open(f"vectorestore/13XHnad8TcZzHtGYKO0X.pkl", "rb") as f:
        VectorStore = pickle.load(f)

C:\Users\Mukesh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm, VectorStore.as_retriever(search_kwargs={"k": 4}), return_source_documents=True,combine_docs_chain_kwargs={"prompt": prompt})

In [12]:
chat_history = []

In [27]:
query = "Revenue of tcs in financial year 2023"
result=chain.invoke({"question": query, "chat_history": chat_history})

In [28]:
text=result['answer']

In [30]:
result["source_documents"]

[Document(page_content='as amended from time to time.  Significant accounting policies used in the preparation of the financial statements are disclosed in the notes to the consolidated  financial statements. The following table gives an overview of the consolidated financial results of the company: (` crore) FY 2023 FY 2022  Amount % of Revenue % Growth Amount % of Revenue Revenue from operations 225,458 100.0 17.6 191,754 100.0 Earnings before interest, tax, depreciation and  amortization (EBITDA) (before other income) 59,259 26.3 11.7 53,057 27.7 Profit Before Tax (PBT) 56,907 25.2 10.1 51,687 27.0 Profit after tax attributable to shareholders of the  company 42,147 18.7 10.0 38,327 20.0 Earnings per share (in `) 115.19 11.2 103.62 Analysis of revenue growth On a reported basis, TCS’ revenue grew 17.6% in FY', metadata={'page': 92}),
 Document(page_content='attributable to shareholders of the  company 42,147 18.7 10.0 38,327 20.0 Earnings per share (in `) 115.19 11.2 103.62 Analysis

In [ ]:
sorDb = FAISS.from_documents(result["source_documents"], embedding=embeddings)

In [32]:
doc= sorDb.similarity_search(result['answer'])

In [36]:
doc[0].metadata["page"]

92

<h1>Retrive check</h1>

In [17]:
query="Revenue of company"

In [18]:
doc=VectorStore.similarity_search(query,k=10)

In [19]:
score =VectorStore.max_marginal_relevance_search(query,k=10)

In [21]:
for page in score:
    print(page.metadata["page"])

84
4
168
167
145
108
84
141
4
179


In [226]:
for page in doc:
    print(page.metadata["page"])

48
91
276
71
205
223
293
72
15
204


In [95]:
result['source_documents'][0].metadata

{'source': 'temp/temp.pdf',
 'file_path': 'temp/temp.pdf',
 'page': 2,
 'total_pages': 346,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': 'Adobe Acrobat Pro 10.0.0',
 'producer': 'Adobe Acrobat Pro 10.0.0',
 'creationDate': "D:20230606223924+05'30'",
 'modDate': "D:20230612105241+05'30'",
 'trapped': ''}

In [8]:
from langchain.storage import InMemoryStore
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings


In [3]:
loader = PyMuPDFLoader("Reports/jio.pdf")
doc = loader.load()

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Mukesh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
retriever.add_documents(doc, ids=None)

ValueError: Batch size 194 exceeds maximum batch size 166